In [39]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy import displacy
from spacy.matcher import PhraseMatcher

def CourseOutcomes(question,course_outcomes):



# Load the NLP model and add the course outcomes and questions to the nlp pipeline
    nlp = spacy.load("en_core_web_sm")
    nlp.add_pipe("sentencizer")

# Define a function to preprocess the text
    def preprocess(text):
        doc = nlp(text)
        tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct and not token.is_space]
        return ' '.join(tokens)

# Preprocess the course outcomes and questions
    course_outcomes = [preprocess(outcome) for outcome in course_outcomes]
    Questions = preprocess(question)

# Define a function to extract keywords
    def extract_keywords(text, nlp, max_keywords=10, min_token_freq=2, min_keyword_len=2):
        doc = nlp(text)
    # Get the frequency of each token and its lemma in the document
        freq = {}
        for token in doc:
            if not token.is_stop and not token.is_punct and not token.is_space:
                for form in (token.lemma_, token.text):
                    if form not in freq:
                        freq[form] = 0
                    freq[form] += 1
    # Sort the tokens by frequency
        sorted_tokens = sorted(freq, key=freq.get, reverse=True)
    # Extract the keywords based on the sorted tokens
        keywords = []
        for token in sorted_tokens:
            if len(token) < min_keyword_len:
                continue
            if freq[token] < min_token_freq or len(keywords) >= max_keywords:
                break
            keywords.append(token)
        return keywords


    def map_questions_to_outcomes(course_outcomes, question, nlp, max_keywords=10, min_token_freq=2, min_keyword_len=2):
        # Preprocess the course outcomes and questions
        course_outcomes = [preprocess(outcome) for outcome in course_outcomes]
        question = preprocess(question) 
    # Extract the keywords for each course outcome
        keywords = {}
        for i, outcome in enumerate(course_outcomes):
            keywords[f'CO{i+1}'] = extract_keywords(outcome, nlp, max_keywords, min_token_freq, min_keyword_len)
    # Create a PhraseMatcher for each course outcome
        matchers = {}
        for key in keywords:
            matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
            patterns = [nlp(keyword) for keyword in keywords[key]]
            matcher.add(key, None, *patterns)
            matchers[key] = matcher
    # Match the keywords from the course outcomes to the questions
        
        question_doc = nlp(question)
        outcome=None
        for key in matchers:
                matches = matchers[key](question_doc)
                if matches:
                   outcome=key
                   break
        return outcome

    
   
    mapped_question = map_questions_to_outcomes(course_outcomes, question, nlp)
        
    return mapped_question


In [43]:
def identify_blooms_level(question,blooms_verbs):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(question)
    verbs = [token.lemma_ for token in doc if token.pos_ == 'VERB']
    for level, bloom_verbs in blooms_verbs.items():
        if any(verb in bloom_verbs for verb in verbs):
            return level
    return None

In [44]:
def Mapper(questions,course_outcomes):
    blooms_verbs = {
    'Remembering': ['define', 'list', 'name', 'recall', 'recognize'],
    'Understanding': ['explain', 'interpret', 'summarize', 'classify', 'compare'],
    'Applying': ['apply', 'solve', 'use', 'demonstrate', 'modify','implement'],
    'Analyzing': ['analyze', 'compare', 'differentiate', 'organize', 'categorize'],
    'Evaluating': ['evaluate', 'judge', 'assess', 'critique', 'justify'],
    'Creating': ['create', 'design', 'develop', 'synthesize', 'construct']
    }
    mapper={}
    for question in questions:
        bloom=identify_blooms_level(question,blooms_verbs)
        Question_local=question
        CO=CourseOutcomes(Question_local,course_outcomes)
        temp={
            "Blooms_level":bloom,
            "CO":CO
        }
        mapper[question]=temp
    return mapper


{'Write a Python program to implement a linked list data structure': {'Blooms_level': 'Applying', 'CO': 'CO2'}, 'What is the time complexity of the quicksort algorithm?': {'Blooms_level': None, 'CO': 'CO2'}, 'How would you communicate a technical concept to a non-technical audience?': {'Blooms_level': None, 'CO': 'CO3'}}
